FusionadorNotebooksColab.py

Script para fusionar múltiples archivos Jupyter Notebook (.ipynb) en uno solo.
Versión adaptada para Google Colab sin interfaz gráfica.
Permite seleccionar archivos locales o descargarlos desde repositorios de GitHub.

In [3]:
import os
import json
import tempfile
import shutil
import zipfile
import re
import argparse
from urllib.parse import urlparse
import requests
from google.colab import files
import git
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

In [4]:
class FusionadorNotebooksColab:
    """Clase para fusionar múltiples archivos .ipynb en uno solo en Google Colab."""

    def __init__(self):
        """Inicializa la aplicación."""
        self.archivosSeleccionados = []
        self.reposSeleccionados = []
        self.nombreArchivoDestino = "cuaderno_fusionado.ipynb"
        self.carpetaTemporal = tempfile.mkdtemp()
        self.archivosSubidos = []
        self.ConfigurarInterfazColab()

    def ConfigurarInterfazColab(self):
        """Configura la interfaz basada en widgets para Google Colab."""
        # Título principal
        display(HTML("<h2>Fusionador de Notebooks para Google Colab</h2>"))

        # Sección para subir archivos
        self.btnSubirArchivos = widgets.Button(
            description='Subir Archivos .ipynb',
            icon='upload',
            button_style='primary',
            tooltip='Haz clic para subir notebooks'
        )
        self.btnSubirArchivos.on_click(self.SubirArchivosLocales)

        # Sección para ingresar repositorios
        self.etiquetaRepo = widgets.HTML(value="<b>Repositorios de GitHub:</b>")
        self.entradaRepo = widgets.Text(
            placeholder='Ingresa la URL del repositorio',
            description='URL:',
            layout=widgets.Layout(width='70%')
        )
        self.btnAgregarRepo = widgets.Button(
            description='Agregar Repositorio',
            icon='plus',
            button_style='info',
            tooltip='Agregar este repositorio a la lista'
        )
        self.btnAgregarRepo.on_click(self.AgregarRepositorio)

        # Sección para mostrar archivos y repositorios seleccionados
        self.etiquetaArchivos = widgets.HTML(value="<b>Archivos seleccionados:</b>")
        self.listaArchivos = widgets.HTML(value="<i>Ningún archivo seleccionado</i>")

        self.etiquetaRepos = widgets.HTML(value="<b>Repositorios seleccionados:</b>")
        self.listaRepos = widgets.HTML(value="<i>Ningún repositorio seleccionado</i>")

        # Nombre del archivo resultante
        self.etiquetaNombre = widgets.HTML(value="<b>Nombre del archivo resultante:</b>")
        self.entradaNombre = widgets.Text(
            value=self.nombreArchivoDestino,
            description='Nombre:',
            layout=widgets.Layout(width='50%')
        )

        # Botones de acción
        self.btnFusionar = widgets.Button(
            description='Fusionar y Descargar',
            icon='save',
            button_style='success',
            tooltip='Fusionar todos los notebooks y descargar'
        )
        self.btnFusionar.on_click(self.FusionarYDescargar)

        self.btnLimpiar = widgets.Button(
            description='Limpiar Todo',
            icon='trash',
            button_style='danger',
            tooltip='Limpiar todas las selecciones'
        )
        self.btnLimpiar.on_click(self.LimpiarTodo)

        # Barra de progreso
        self.barraProgreso = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Progreso:',
            bar_style='info',
            orientation='horizontal'
        )

        # Mensaje de estado
        self.etiquetaEstado = widgets.HTML(value="<i>Listo para comenzar</i>")

        # Mostrar todo en la interfaz
        display(self.btnSubirArchivos)
        display(widgets.HBox([self.entradaRepo, self.btnAgregarRepo]))
        display(self.etiquetaArchivos)
        display(self.listaArchivos)
        display(self.etiquetaRepos)
        display(self.listaRepos)
        display(self.entradaNombre)
        display(widgets.HBox([self.btnFusionar, self.btnLimpiar]))
        display(self.barraProgreso)
        display(self.etiquetaEstado)

    def SubirArchivosLocales(self, b):
        """Permite al usuario subir archivos .ipynb locales a Colab."""
        self.ActualizarEstado("Subiendo archivos...")

        try:
            archivosSubidos = files.upload()

            for nombreArchivo, contenido in archivosSubidos.items():
                if nombreArchivo.endswith('.ipynb'):
                    rutaTemporal = os.path.join(self.carpetaTemporal, nombreArchivo)

                    with open(rutaTemporal, 'wb') as f:
                        f.write(contenido)

                    self.archivosSeleccionados.append(rutaTemporal)
                    self.archivosSubidos.append(nombreArchivo)

            # Actualizar la lista de archivos mostrada
            self.ActualizarListaArchivos()
            self.ActualizarEstado(f"Se han subido {len(archivosSubidos)} archivo(s)")

        except Exception as e:
            self.ActualizarEstado(f"Error al subir archivos: {str(e)}")

    def ActualizarListaArchivos(self):
        """Actualiza la lista de archivos mostrada en la interfaz."""
        if self.archivosSubidos:
            listaHTML = "<ul>"
            for archivo in self.archivosSubidos:
                listaHTML += f"<li>{archivo}</li>"
            listaHTML += "</ul>"
            self.listaArchivos.value = listaHTML
        else:
            self.listaArchivos.value = "<i>Ningún archivo seleccionado</i>"

    def ActualizarListaRepos(self):
        """Actualiza la lista de repositorios mostrada en la interfaz."""
        if self.reposSeleccionados:
            listaHTML = "<ul>"
            for repo in self.reposSeleccionados:
                listaHTML += f"<li>{repo}</li>"
            listaHTML += "</ul>"
            self.listaRepos.value = listaHTML
        else:
            self.listaRepos.value = "<i>Ningún repositorio seleccionado</i>"

    def AgregarRepositorio(self, b):
        """Agrega un repositorio de GitHub a la lista."""
        urlRepo = self.entradaRepo.value.strip()

        if not urlRepo:
            self.ActualizarEstado("⚠️ Por favor ingresa la URL del repositorio")
            return

        if not self.EsUrlGitHubValida(urlRepo):
            self.ActualizarEstado("⚠️ Por favor ingresa una URL de GitHub válida")
            return

        if urlRepo not in self.reposSeleccionados:
            self.reposSeleccionados.append(urlRepo)
            self.entradaRepo.value = ""
            self.ActualizarListaRepos()
            self.ActualizarEstado(f"Repositorio agregado: {urlRepo}")

    def EsUrlGitHubValida(self, url):
        """Verifica si la URL proporcionada es una URL válida de GitHub."""
        patron = r'^https?://(?:www\.)?github\.com/[\w-]+/[\w.-]+/?.*$'
        return bool(re.match(patron, url))

    def LimpiarTodo(self, b):
        """Limpia todas las selecciones."""
        self.archivosSeleccionados = []
        self.archivosSubidos = []
        self.reposSeleccionados = []
        self.ActualizarListaArchivos()
        self.ActualizarListaRepos()
        self.entradaNombre.value = "cuaderno_fusionado.ipynb"
        self.ActualizarEstado("Se ha limpiado todo")

    def ActualizarEstado(self, mensaje):
        """Actualiza el mensaje de estado en la interfaz."""
        self.etiquetaEstado.value = f"<i>{mensaje}</i>"

    def ActualizarProgreso(self, valor):
        """Actualiza la barra de progreso."""
        self.barraProgreso.value = valor

    def DescargarRepositorio(self, url):
        """Descarga los archivos .ipynb de un repositorio de GitHub."""
        self.ActualizarEstado(f"Descargando repositorio: {url}")

        # Crear un directorio temporal para el repositorio
        dirRepo = os.path.join(self.carpetaTemporal, self.ObtenerNombreRepo(url))

        try:
            # Clonar el repositorio
            git.Repo.clone_from(url, dirRepo, depth=1)

            # Buscar todos los archivos .ipynb en el repositorio
            archivosIpynb = []
            for raiz, dirs, archivos in os.walk(dirRepo):
                for archivo in archivos:
                    if archivo.endswith('.ipynb'):
                        rutaCompleta = os.path.join(raiz, archivo)
                        archivosIpynb.append(rutaCompleta)

            self.ActualizarEstado(f"Se encontraron {len(archivosIpynb)} notebooks en el repositorio")
            return archivosIpynb

        except Exception as e:
            self.ActualizarEstado(f"❌ Error al descargar el repositorio: {str(e)}")
            return []

    def ObtenerNombreRepo(self, url):
        """Extrae el nombre del repositorio de una URL de GitHub."""
        partes = urlparse(url).path.strip('/').split('/')
        if len(partes) >= 2:
            return f"{partes[0]}_{partes[1]}"
        return "repo_desconocido"

    def FusionarCuadernos(self, archivos):
        """Fusiona múltiples archivos .ipynb en uno solo."""
        if not archivos:
            return None

        self.ActualizarEstado("Fusionando cuadernos...")

        # Usar el primer archivo como base
        try:
            with open(archivos[0], 'r', encoding='utf-8') as f:
                cuadernoBase = json.load(f)

            # Si hay más archivos, agregar sus celdas al cuaderno base
            for i, rutaArchivo in enumerate(archivos[1:], 1):
                self.ActualizarProgreso(i * 100 // len(archivos))

                try:
                    with open(rutaArchivo, 'r', encoding='utf-8') as f:
                        cuaderno = json.load(f)

                    # Agregar un marcador para indicar el inicio de un nuevo archivo
                    cuadernoBase['cells'].append({
                        "cell_type": "markdown",
                        "metadata": {},
                        "source": [f"## Contenido de: {os.path.basename(rutaArchivo)}"]
                    })

                    # Agregar las celdas del cuaderno actual
                    cuadernoBase['cells'].extend(cuaderno['cells'])

                except Exception as e:
                    self.ActualizarEstado(f"Error al procesar {os.path.basename(rutaArchivo)}: {str(e)}")

            return cuadernoBase

        except Exception as e:
            self.ActualizarEstado(f"❌ Error al fusionar cuadernos: {str(e)}")
            return None

    def FusionarYDescargar(self, b):
        """Fusiona todos los cuadernos seleccionados y permite descargar el resultado."""
        # Actualizar el nombre del archivo destino
        nombreDestino = self.entradaNombre.value.strip()
        if not nombreDestino:
            nombreDestino = "cuaderno_fusionado.ipynb"
        elif not nombreDestino.endswith('.ipynb'):
            nombreDestino += '.ipynb'

        self.nombreArchivoDestino = nombreDestino

        # Verificar si hay archivos para fusionar
        if not self.archivosSeleccionados and not self.reposSeleccionados:
            self.ActualizarEstado("⚠️ No hay archivos ni repositorios seleccionados para fusionar")
            return

        # Comenzar la barra de progreso
        self.ActualizarProgreso(0)
        self.ActualizarEstado("Iniciando proceso de fusión...")

        # Lista para almacenar todos los archivos .ipynb
        todosLosArchivos = self.archivosSeleccionados.copy()

        # Descargar archivos de repositorios si hay alguno seleccionado
        if self.reposSeleccionados:
            for i, repo in enumerate(self.reposSeleccionados):
                self.ActualizarProgreso((i + 1) * 50 // len(self.reposSeleccionados))
                archivosRepo = self.DescargarRepositorio(repo)
                todosLosArchivos.extend(archivosRepo)

        # Fusionar los cuadernos
        cuadernoFusionado = self.FusionarCuadernos(todosLosArchivos)

        if cuadernoFusionado:
            # Guardar el resultado en un archivo temporal
            rutaTemporal = os.path.join(self.carpetaTemporal, self.nombreArchivoDestino)
            with open(rutaTemporal, 'w', encoding='utf-8') as f:
                json.dump(cuadernoFusionado, f, ensure_ascii=False, indent=2)

            # Descargar el archivo fusionado
            try:
                self.ActualizarEstado(f"Descargando el archivo fusionado: {self.nombreArchivoDestino}")
                files.download(rutaTemporal)
                self.ActualizarEstado(f"✅ Cuaderno fusionado descargado como: {self.nombreArchivoDestino}")
            except Exception as e:
                self.ActualizarEstado(f"❌ Error al descargar: {str(e)}")

        # Completar la barra de progreso
        self.ActualizarProgreso(100)

In [5]:
def main():
    """Función principal que inicia la aplicación."""
    app = FusionadorNotebooksColab()
    return app

In [6]:
if __name__ == "__main__":
    main()

Button(button_style='primary', description='Subir Archivos .ipynb', icon='upload', style=ButtonStyle(), toolti…

HTML(value='<b>Archivos seleccionados:</b>')

HTML(value='<i>Ningún archivo seleccionado</i>')

HTML(value='<b>Repositorios seleccionados:</b>')

HTML(value='<i>Ningún repositorio seleccionado</i>')

Text(value='cuaderno_fusionado.ipynb', description='Nombre:', layout=Layout(width='50%'))

IntProgress(value=0, bar_style='info', description='Progreso:')

HTML(value='<i>Listo para comenzar</i>')